<a href="https://colab.research.google.com/github/wallmosh/masakhane/blob/master/Hausa_English_NMT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:

from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# TODO: Set your source and target languages. Keep in mind, these traditionally use language codes as found here:
# These will also become the suffix's of all vocab and corpus files used throughout
import os
source_language = "ha"
target_language = "en" 
lc = False  # If True, lowercase the data.
seed = 42  # Random seed for shuffling.
tag = "baseline" # Give a unique name to your folder - this is to ensure you don't rewrite any models you've already submitted

os.environ["src"] = source_language # Sets them in bash as well, since we often use bash scripts
os.environ["tgt"] = target_language
os.environ["tag"] = tag

# # This will save it to a folder in our gdrive instead!
# !mkdir -p "/content/drive/My Drive/masakhane/$src-$tgt-$tag"
# os.environ["gdrive_path"] = "/content/drive/My Drive/masakhane/%s-%s-%s" % (source_language, target_language, tag)

# This will save it to a folder in our gdrive instead! 
!mkdir -p "/content/drive/My Drive/masakhane/$src-$tgt-$tag"
g_drive_path = "/content/drive/My Drive/masakhane/%s-%s-%s" % (source_language, target_language, tag)
os.environ["gdrive_path"] = g_drive_path
models_path = '%s/models/%s%s_transformer'% (g_drive_path, source_language, target_language)
# model temporary directory for training
model_temp_dir = "/content/drive/My Drive/masakhane/model-temp"
# model permanent storage on the drive
!mkdir -p "$gdrive_path/models/${src}${tgt}_transformer/"

In [4]:
!echo $gdrive_path

/content/drive/My Drive/masakhane/ha-en-baseline


In [5]:
# Install opus-tools
! pip install opustools-pkg

     |████████████████████████████████| 81kB 4.8MB/s 


In [6]:

# Downloading our corpus
! opus_read -d JW300 -s $src -t $tgt -wm moses -w jw300.$src jw300.$tgt -q

# extract the corpus file
! gunzip JW300_latest_xml_$tgt-$src.xml.gz


Alignment file /proj/nlpl/data/OPUS/JW300/latest/xml/en-ha.xml.gz not found. The following files are available for downloading:

   2 MB https://object.pouta.csc.fi/OPUS-JW300/v1/xml/en-ha.xml.gz
 263 MB https://object.pouta.csc.fi/OPUS-JW300/v1/xml/en.zip
  24 MB https://object.pouta.csc.fi/OPUS-JW300/v1/xml/ha.zip

 289 MB Total size
./JW300_latest_xml_en-ha.xml.gz ... 100% of 2 MB
./JW300_latest_xml_en.zip ... 100% of 263 MB
./JW300_latest_xml_ha.zip ... 100% of 24 MB


In [7]:

import pandas as pd

# TMX file to dataframe
source_file = 'jw300.' + source_language
target_file = 'jw300.' + target_language

source = []
target = []
skip_lines = []  # Collect the line numbers of the source portion to skip the same lines for the target portion.
with open(source_file) as f:
    for i, line in enumerate(f):
        # Skip sentences that are contained in the test set.
        # if line.strip() not in en_test_sents:
        source.append(line.strip())
        # else:
        #     skip_lines.append(i)             
with open(target_file) as f:
    for j, line in enumerate(f):
        # Only add to corpus if corresponding source was not skipped.
        # if j not in skip_lines:
        target.append(line.strip())
    
print('Loaded data and skipped {}/{} lines since contained in test set.'.format(len(skip_lines), i))
    
#df = pd.DataFrame(zip(source_file, target_file), columns=['source_sentence', 'target_sentence'])
# if you get TypeError: data argument can't be an iterator is because of your zip version run this below
df1= pd.DataFrame(list(zip(source, target)), columns=['source_sentence', 'target_sentence'])
print(df1.shape)
df1.head(3)


Loaded data and skipped 0/237064 lines since contained in test set.
(237065, 2)


,source_sentence,target_sentence
0,Tambayoyi Daga Masu Karatu,Questions From Readers
1,Yaya ya kamata Kiristoci su ɗauki riƙo ?,How seriously should Christians view an engage...
2,"Riƙo dalili ne na farinciki , amma kuma muhimm...",An engagement to marry is a cause for happines...


In [8]:

# Downloading our corpus
! opus_read -d Tatoeba -s $src -t $tgt -wm moses -w tatoeba.$src tatoeba.$tgt -q

# extract the corpus file
! gunzip Tatoeba_latest_xml_$tgt-$src.xml.gz


Alignment file /proj/nlpl/data/OPUS/Tatoeba/latest/xml/en-ha.xml.gz not found. The following files are available for downloading:

   4 KB https://object.pouta.csc.fi/OPUS-Tatoeba/v20190709/xml/en-ha.xml.gz
  90 MB https://object.pouta.csc.fi/OPUS-Tatoeba/v20190709/xml/en.zip
  12 KB https://object.pouta.csc.fi/OPUS-Tatoeba/v20190709/xml/ha.zip

  90 MB Total size
./Tatoeba_latest_xml_en-ha.xml.gz ... 100% of 4 KB
./Tatoeba_latest_xml_en.zip ... 100% of 90 MB
./Tatoeba_latest_xml_ha.zip ... 100% of 12 KB


In [9]:
import pandas as pd

# TMX file to dataframe
source_file = 'tatoeba.' + source_language
target_file = 'tatoeba.' + target_language

source = []
target = []
skip_lines = []  # Collect the line numbers of the source portion to skip the same lines for the target portion.
with open(source_file) as f:
    for i, line in enumerate(f):
        # Skip sentences that are contained in the test set.
        # if line.strip() not in en_test_sents:
        source.append(line.strip())
        # else:
        #     skip_lines.append(i)             
with open(target_file) as f:
    for j, line in enumerate(f):
        # Only add to corpus if corresponding source was not skipped.
        # if j not in skip_lines:
        target.append(line.strip())
    
print('Loaded data and skipped {}/{} lines since contained in test set.'.format(len(skip_lines), i))
    
#df = pd.DataFrame(zip(source_file, target_file), columns=['source_sentence', 'target_sentence'])
# if you get TypeError: data argument can't be an iterator is because of your zip version run this below
df2= pd.DataFrame(list(zip(source, target)), columns=['source_sentence', 'target_sentence'])
print(df2.shape)
df2.head(3)

Loaded data and skipped 0/55 lines since contained in test set.
(56, 2)


,source_sentence,target_sentence
0,Kwarai kuwa zaka iya daukarsa idan kanaso .,Of course you can take it if you want .
1,Ka taɓa zuwa Paris ?,Have you ever been to Paris ?
2,Ina ne wajen bahaya ?,Where is the toilet ?


In [10]:
# Downloading our corpus
! opus_read -d wikimedia -s $src -t $tgt -wm moses -w wikimedia.$src wikimedia.$tgt -q

# extract the corpus file
! gunzip wikimedia_latest_xml_$tgt-$src.xml.gz


Alignment file /proj/nlpl/data/OPUS/wikimedia/latest/xml/en-ha.xml.gz not found. The following files are available for downloading:

  20 KB https://object.pouta.csc.fi/OPUS-wikimedia/v20190628/xml/en-ha.xml.gz
  60 MB https://object.pouta.csc.fi/OPUS-wikimedia/v20190628/xml/en.zip
 252 KB https://object.pouta.csc.fi/OPUS-wikimedia/v20190628/xml/ha.zip

  60 MB Total size
./wikimedia_latest_xml_en-ha.xml.gz ... 100% of 20 KB
./wikimedia_latest_xml_en.zip ... 100% of 60 MB
./wikimedia_latest_xml_ha.zip ... 100% of 252 KB


In [11]:

# TMX file to dataframe
source_file = 'wikimedia.' + source_language
target_file = 'wikimedia.' + target_language

source = []
target = []
skip_lines = []  # Collect the line numbers of the source portion to skip the same lines for the target portion.
with open(source_file) as f:
    for i, line in enumerate(f):
        # Skip sentences that are contained in the test set.
        # if line.strip() not in en_test_sents:
        source.append(line.strip())
        # else:
        #     skip_lines.append(i)             
with open(target_file) as f:
    for j, line in enumerate(f):
        # Only add to corpus if corresponding source was not skipped.
        # if j not in skip_lines:
        target.append(line.strip())
    
print('Loaded data and skipped {}/{} lines since contained in test set.'.format(len(skip_lines), i))
    
#df = pd.DataFrame(zip(source_file, target_file), columns=['source_sentence', 'target_sentence'])
# if you get TypeError: data argument can't be an iterator is because of your zip version run this below
df3= pd.DataFrame(list(zip(source, target)), columns=['source_sentence', 'target_sentence'])
print(df3.shape)
df3.head(3)

Loaded data and skipped 0/702 lines since contained in test set.
(703, 2)


,source_sentence,target_sentence
0,References,References
1,"↑ "" Will Smith Net Worth - atlantablackstar "" ...","↑ "" Will Smith Net Worth - atlantablackstar "" ..."
2,"Willard Carroll "" Will "" Smith Jr . [ 2 ] [ 3 ...","Willard Carroll "" Will "" Smith Jr.[2][3][4 ] (..."


In [12]:
# Downloading our corpus
! opus_read -d Tanzil -s $src -t $tgt -wm moses -w tanzil.$src tanzil.$tgt -q

# extract the corpus file
! gunzip Tanzil_latest_xml_$tgt-$src.xml.gz


Alignment file /proj/nlpl/data/OPUS/Tanzil/latest/xml/en-ha.xml.gz not found. The following files are available for downloading:

   1 MB https://object.pouta.csc.fi/OPUS-Tanzil/v1/xml/en-ha.xml.gz
  34 MB https://object.pouta.csc.fi/OPUS-Tanzil/v1/xml/en.zip
   1 MB https://object.pouta.csc.fi/OPUS-Tanzil/v1/xml/ha.zip

  36 MB Total size
./Tanzil_latest_xml_en-ha.xml.gz ... 100% of 1 MB
./Tanzil_latest_xml_en.zip ... 100% of 34 MB
./Tanzil_latest_xml_ha.zip ... 100% of 1 MB


In [13]:

# TMX file to dataframe
source_file = 'tanzil.' + source_language
target_file = 'tanzil.' + target_language

source = []
target = []
skip_lines = []  # Collect the line numbers of the source portion to skip the same lines for the target portion.
with open(source_file) as f:
    for i, line in enumerate(f):
        # Skip sentences that are contained in the test set.
        # if line.strip() not in en_test_sents:
        source.append(line.strip())
        # else:
        #     skip_lines.append(i)             
with open(target_file) as f:
    for j, line in enumerate(f):
        # Only add to corpus if corresponding source was not skipped.
        # if j not in skip_lines:
        target.append(line.strip())
    
print('Loaded data and skipped {}/{} lines since contained in test set.'.format(len(skip_lines), i))
    
#df = pd.DataFrame(zip(source_file, target_file), columns=['source_sentence', 'target_sentence'])
# if you get TypeError: data argument can't be an iterator is because of your zip version run this below
df4= pd.DataFrame(list(zip(source, target)), columns=['source_sentence', 'target_sentence'])
print(df4.shape)
df4.head(3)

Loaded data and skipped 0/132100 lines since contained in test set.
(132101, 2)


,source_sentence,target_sentence
0,"Da sũnan Allah , Mai rahama , Mai jin ƙai .","In the name of Allah , most benevolent , ever-..."
1,"Godiya ta tabbata ga Allah , Ubangijin halittu ;","ALL PRAISE BE to Allah , Lord of all the worlds ,"
2,"Mai rahama , Mai jin ƙai ;","Most beneficent , ever-merciful ,"


In [0]:
List = [df1, df2, df3, df4]  # List of your dataframes
df = pd.concat(List)

In [15]:
df.shape

(369925, 2)

In [0]:

# drop duplicate translations
df_pp = df.drop_duplicates()

# drop conflicting translations
# (this is optional and something that you might want to comment out 
# depending on the size of your corpus)
# df_pp.drop_duplicates(subset='source_sentence', inplace=True)
# df_pp.drop_duplicates(subset='target_sentence', inplace=True)

# Shuffle the data to remove bias in dev set selection.
df_pp = df_pp.sample(frac=1, random_state=seed).reset_index(drop=True)

In [17]:
df_pp.shape

(351024, 2)

In [0]:
# # Install fuzzy wuzzy to remove "almost duplicate" sentences in the
# # test and training sets.
# ! pip install fuzzywuzzy
# ! pip install python-Levenshtein
# import time
# from fuzzywuzzy import process
# import numpy as np

# # reset the index of the training set after previous filtering
# df_pp.reset_index(drop=False, inplace=True)

# # Remove samples from the training data set if they "almost overlap" with the
# # samples in the test set.

# # Filtering function. Adjust pad to narrow down the candidate matches to
# # within a certain length of characters of the given sample.
# def fuzzfilter(sample, candidates, pad):
#   candidates = [x for x in candidates if len(x) <= len(sample)+pad and len(x) >= len(sample)-pad] 
#   if len(candidates) > 0:
#     return process.extractOne(sample, candidates)[1]
#   else:
#     return np.nan

# # NOTE - This might run slow depending on the size of your training set. We are
# # printing some information to help you track how long it would take. 
# scores = []
# start_time = time.time()
# for idx, row in df_pp.iterrows():
#   scores.append(fuzzfilter(row['source_sentence'], list(en_test_sents), 5))
#   if idx % 1000 == 0:
#     hours, rem = divmod(time.time() - start_time, 3600)
#     minutes, seconds = divmod(rem, 60)
#     print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds), "%0.2f percent complete" % (100.0*float(idx)/float(len(df_pp))))

# # Filter out "almost overlapping samples"
# df_pp['scores'] = scores
# df_pp = df_pp[df_pp['scores'] < 95]

In [19]:
# This section does the split between train/dev for the parallel corpora then saves them as separate files
# We use 1000 dev test and the given test set.
import csv
import numpy as np

# Do the split between dev/train and create parallel corpora
num_dev_patterns = 1000

# Optional: lower case the corpora - this will make it easier to generalize, but without proper casing.
if lc:  # Julia: making lowercasing optional
    df_pp["source_sentence"] = df_pp["source_sentence"].str.lower()
    df_pp["target_sentence"] = df_pp["target_sentence"].str.lower()

#Divide df_pp into train and test sets

msk = np.random.rand(len(df_pp)) < 0.98
train = df_pp[msk]
test_df = df_pp[~msk]

# Further divide train into train and additional dev set which should be the last 1000
dev_df = train.tail(num_dev_patterns) # Herman: Error in original
stripped_df = train.drop(train.tail(num_dev_patterns).index)

with open("test."+source_language, "w") as src_file, open("test."+target_language, "w") as trg_file:
  for index, row in test_df.iterrows():
    src_file.write(row["source_sentence"]+"\n")
    trg_file.write(row["target_sentence"]+"\n")

with open("train."+source_language, "w") as src_file, open("train."+target_language, "w") as trg_file:
  for index, row in stripped_df.iterrows():
    src_file.write(row["source_sentence"]+"\n")
    trg_file.write(row["target_sentence"]+"\n")
    
with open("dev."+source_language, "w") as src_file, open("dev."+target_language, "w") as trg_file:
  for index, row in dev_df.iterrows():
    src_file.write(row["source_sentence"]+"\n")
    trg_file.write(row["target_sentence"]+"\n")

#stripped[["source_sentence"]].to_csv("train."+source_language, header=False, index=False)  # Herman: Added `header=False` everywhere
#stripped[["target_sentence"]].to_csv("train."+target_language, header=False, index=False)  # Julia: Problematic handling of quotation marks.

#dev[["source_sentence"]].to_csv("dev."+source_language, header=False, index=False)
#dev[["target_sentence"]].to_csv("dev."+target_language, header=False, index=False)

# Doublecheck the format below. There should be no extra quotation marks or weird characters.
! head train.*
! head dev.*
! head test.*

==> train.en <==
And We have placed in the earth firm mountains lest it should move away with them , and We placed therein Passages for paths , that haply they may be guided .
“ YOU saw a lot of casual clothing , especially when it was hot , ” reported a Dutch newspaper regarding a meeting of church leaders .
How will God relieve mankind of suffering ?
Jesus felt compassion for the people he preached to , and we should likewise be moved by compassion to preach to others . ​ — Matthew 9 : 36 .
Every soul shall have a taste of death : And only on the Day of Judgment shall you be paid your full recompense .
And We had already tried before them the people of Pharaoh , and there came to them a noble messenger ,
He was an instructor at the Bible School of Gilead , and later he served as a member of the Governing Body .
God had in mind for their children to have children , until the human family filled the whole earth .
Reading the Bible daily and meditating on what we read can help to guard 

In [0]:
stripped_df.to_csv('train.csv')
!cp train.csv drive/My\ Drive/masakhane/

In [0]:
test_df.to_csv('test.csv')
!cp test.csv drive/My\ Drive/masakhane/

In [0]:
dev_df.to_csv('dev.csv')
!cp dev.csv drive/My\ Drive/masakhane/

In [23]:

# Install JoeyNMT
! git clone https://github.com/joeynmt/joeynmt.git
! cd joeynmt; pip3 install .

Cloning into 'joeynmt'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 2380 (delta 0), reused 0 (delta 0), pack-reused 2377
Receiving objects: 100% (2380/2380), 2.60 MiB | 4.86 MiB/s, done.
Resolving deltas: 100% (1670/1670), done.
Processing /content/joeynmt
     |████████████████████████████████| 61kB 4.7MB/s 
     |████████████████████████████████| 276kB 8.5MB/s 
     |████████████████████████████████| 307kB 17.2MB/s 
     |████████████████████████████████| 17.1MB 205kB/s 
     |████████████████████████████████| 51kB 8.2MB/s 
     |████████████████████████████████| 215kB 55.1MB/s 
     |████████████████████████████████| 747kB 54.8MB/s 
     |████████████████████████████████| 61kB 10.4MB/s 
  Created wheel for joeynmt: filename=joeynmt-0.0.1-cp36-none-any.whl size=73768 sha256=9a925354abe22a4d4ef77935052fc78c43f56060c8bf622878f9c8d3ab603ae9
  Stored in directory: /tmp/pip-ephem-wheel-cac

In [24]:

# One of the huge boosts in NMT performance was to use a different method of tokenizing. 
# Usually, NMT would tokenize by words. However, using a method called BPE gave amazing boosts to performance

# Do subword NMT
from os import path
os.environ["src"] = source_language # Sets them in bash as well, since we often use bash scripts
os.environ["tgt"] = target_language

# Learn BPEs on the training data.
os.environ["data_path"] = path.join("joeynmt", "data", source_language + target_language) # Herman! 
! subword-nmt learn-joint-bpe-and-vocab --input train.$src train.$tgt -s 4000 -o bpe.codes.4000 --write-vocabulary vocab.$src vocab.$tgt

# Apply BPE splits to the development and test data.
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < train.$src > train.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < train.$tgt > train.bpe.$tgt

! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < dev.$src > dev.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < dev.$tgt > dev.bpe.$tgt
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < test.$src > test.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < test.$tgt > test.bpe.$tgt

# Create directory, move everyone we care about to the correct location
! mkdir -p $data_path
! cp train.* $data_path
! cp test.* $data_path
! cp dev.* $data_path
! cp bpe.codes.4000 $data_path
! ls $data_path

# Also move everything we care about to a mounted location in google drive (relevant if running in colab) at gdrive_path
! cp train.* "$gdrive_path"
! cp test.* "$gdrive_path"
! cp dev.* "$gdrive_path"
! cp bpe.codes.4000 "$gdrive_path"
! ls "$gdrive_path"

# Create that vocab using build_vocab
! sudo chmod 777 joeynmt/scripts/build_vocab.py
! joeynmt/scripts/build_vocab.py joeynmt/data/$src$tgt/train.bpe.$src joeynmt/data/$src$tgt/train.bpe.$tgt --output_path "$gdrive_path/vocab.txt"

# Some output
! echo "BPE Hausa Sentences"
! tail -n 5 test.bpe.$tgt
! echo "Combined BPE Vocab"
! tail -n 10 joeynmt/data/$src$tgt/vocab.txt  

bpe.codes.4000	dev.csv  test.bpe.en  test.en	    train.bpe.ha  train.ha
dev.bpe.en	dev.en	 test.bpe.ha  test.ha	    train.csv
dev.bpe.ha	dev.ha	 test.csv     train.bpe.en  train.en
bpe.codes.4000	dev.csv  models       test.csv	train.bpe.en  train.en
dev.bpe.en	dev.en	 test.bpe.en  test.en	train.bpe.ha  train.ha
dev.bpe.ha	dev.ha	 test.bpe.ha  test.ha	train.csv     vocab.txt
BPE Hausa Sentences
As we were un@@ lo@@ ad@@ ing our things , we over@@ heard her say on the t@@ el@@ e@@ ph@@ one , “ The children are here . ”
How can God’s spirit help us when problems see@@ m over@@ wh@@ el@@ ming ?
But whatever the out@@ come of such c@@ as@@ es , the coura@@ ge God’s servants dis@@ play under tri@@ al makes God rejo@@ ice .
Ju@@ st as the woman in the illustr@@ ation us@@ es help@@ ful to@@ ol@@ s to ac@@ com@@ pl@@ ish her tas@@ k , so today we have to@@ ol@@ s that help us to ac@@ com@@ pl@@ ish our God - given responsib@@ ility to as@@ sis@@ t those who are wea@@ k .
W@@ ith@@ in a sh@@ or

In [0]:
# # One of the huge boosts in NMT performance was to use a different method of tokenizing. 
# # Usually, NMT would tokenize by words. However, using a method called BPE gave amazing boosts to performance

# # Do subword NMT
# from os import path
# os.environ["src"] = source_language # Sets them in bash as well, since we often use bash scripts
# os.environ["tgt"] = target_language

# # Learn BPEs on the training data.
# os.environ["data_path"] = path.join("joeynmt", "data", source_language + target_language) # Herman! 
# ! subword-nmt learn-joint-bpe-and-vocab --input train.$src train.$tgt -s 4000 -o bpe.codes.4000 --write-vocabulary vocab.$src vocab.$tgt

# # Apply BPE splits to the development and test data.
# ! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < train.$src > train.bpe.$src
# ! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < train.$tgt > train.bpe.$tgt

# ! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < dev.$src > dev.bpe.$src
# ! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < dev.$tgt > dev.bpe.$tgt
# ! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < test.$src > test.bpe.$src
# ! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < test.$tgt > test.bpe.$tgt

# # Create directory, move everyone we care about to the correct location
# ! mkdir -p $data_path
# ! cp train.* $data_path
# ! cp test.* $data_path
# ! cp dev.* $data_path
# ! cp bpe.codes.4000 $data_path
# ! ls $data_path

# # Also move everything we care about to a mounted location in google drive (relevant if running in colab) at gdrive_path
# ! cp train.* "$gdrive_path"
# ! cp test.* "$gdrive_path"
# ! cp dev.* "$gdrive_path"
# ! cp bpe.codes.4000 "$gdrive_path"
# ! ls "$gdrive_path"

# # Create that vocab using build_vocab
# ! sudo chmod 777 joeynmt/scripts/build_vocab.py
# ! joeynmt/scripts/build_vocab.py joeynmt/data/$src$tgt/train.bpe.$src joeynmt/data/$src$tgt/train.bpe.$tgt --output_path joeynmt/data/$src$tgt/vocab.txt

# # Some output
# ! echo "BPE Hausa Sentences"
# ! tail -n 5 test.bpe.$tgt
# ! echo "Combined BPE Vocab"
# ! tail -n 10 joeynmt/data/$src$tgt/vocab.txt  # Herman

In [0]:
def get_last_checkpoint(directory):
  last_checkpoint = ''
  try:
    for filename in os.listdir(directory):
      if 'best' in filename and filename.endswith(".ckpt"):
        return filename
      if not 'best' in filename and filename.endswith(".ckpt"):
          if not last_checkpoint or int(filename.split('.')[0]) > int(last_checkpoint.split('.')[0]):
            last_checkpoint = filename
  except FileNotFoundError as e:
    print('Error Occur ', e)
  return last_checkpoint

In [26]:
# Copy the created models from the temporary storage to main storage on google drive for persistant storage 
# the content of te folder will be overwrite when you start trainin
!cp -r "/content/drive/My Drive/masakhane/model-temp/"* "$gdrive_path/models/${src}${tgt}_transformer/"
last_checkpoint = get_last_checkpoint(models_path)
print('Last checkpoint :',last_checkpoint)

Last checkpoint : best.ckpt


In [0]:
# # This creates the config file for our JoeyNMT system. It might seem overwhelming so we've provided a couple of useful parameters you'll need to update
# # (You can of course play with all the parameters if you'd like!)

# name = '%s%s' % (source_language, target_language)
# gdrive_path = os.environ["gdrive_path"]

# # Create the config
# config = """
# name: "{name}_transformer"

# data:
#     src: "{source_language}"
#     trg: "{target_language}"
#     train: "{gdrive_path}/train.bpe"
#     dev:   "{gdrive_path}/dev.bpe"
#     test:  "{gdrive_path}/test.bpe"
#     level: "bpe"
#     lowercase: False
#     max_sent_length: 100
#     src_vocab: "{gdrive_path}/vocab.txt"
#     trg_vocab: "{gdrive_path}/vocab.txt"

# testing:
#     beam_size: 10#5
#     alpha: 1.0

# training:
#     #load_model: "{gdrive_path}/models/{name}_transformer/{last_checkpoint}" # uncommented to load a pre-trained model from last checkpoint
#     random_seed: 42
#     optimizer: "adam"
#     normalization: "tokens"
#     adam_betas: [0.9, 0.999] 
#     scheduling: "plateau"           # TODO: try switching from plateau to Noam scheduling
#     patience: 5                     # For plateau: decrease learning rate by decrease_factor if validation score has not improved for this many validation rounds.
#     learning_rate_factor: 0.5       # factor for Noam scheduler (used with Transformer)
#     learning_rate_warmup: 1000      # warmup steps for Noam scheduler (used with Transformer)
#     decrease_factor: 0.7
#     loss: "crossentropy"
#     learning_rate: 0.0003
#     learning_rate_min: 0.00000001
#     weight_decay: 0.0
#     label_smoothing: 0.1
#     batch_size: 4096
#     batch_type: "token"
#     eval_batch_size: 3600
#     eval_batch_type: "token"
#     batch_multiplier: 1
#     early_stopping_metric: "ppl"
#     epochs: 1                   # TODO: Decrease for when playing around and checking of working. Around 30 is sufficient to check if its working at all
#     validation_freq: 1000          # TODO: Set to at least once per epoch.
#     logging_freq: 100
#     eval_metric: "bleu"
#     model_dir: "{model_temp_dir}"
#     overwrite: True               # TODO: Set to True if you want to overwrite possibly existing models. 
#     shuffle: True
#     use_cuda: True
#     max_output_length: 100
#     print_valid_sents: [0, 1, 2, 3]
#     keep_last_ckpts: 3

# model:
#     initializer: "xavier"
#     bias_initializer: "zeros"
#     init_gain: 1.0
#     embed_initializer: "xavier"
#     embed_init_gain: 1.0
#     tied_embeddings: True
#     tied_softmax: True
#     encoder:
#         type: "transformer"
#         num_layers: 6
#         num_heads: 4             # TODO: Increase to 8 for larger data.
#         embeddings:
#             embedding_dim: 256   # TODO: Increase to 512 for larger data.
#             scale: True
#             dropout: 0.2
#         # typically ff_size = 4 x hidden_size
#         hidden_size: 256         # TODO: Increase to 512 for larger data.
#         ff_size: 1024            # TODO: Increase to 2048 for larger data.
#         dropout: 0.3
#     decoder:
#         type: "transformer"
#         num_layers: 6
#         num_heads: 4              # TODO: Increase to 8 for larger data.
#         embeddings:
#             embedding_dim: 256    # TODO: Increase to 512 for larger data.
#             scale: True
#             dropout: 0.2
#         # typically ff_size = 4 x hidden_size
#         hidden_size: 256         # TODO: Increase to 512 for larger data.
#         ff_size: 1024            # TODO: Increase to 2048 for larger data.
#         dropout: 0.3
# """.format(name=name, gdrive_path=os.environ["gdrive_path"], source_language=source_language, target_language=target_language, model_temp_dir=model_temp_dir, last_checkpoint=last_checkpoint)
# with open("joeynmt/configs/transformer_{name}.yaml".format(name=name),'w') as f:
#     f.write(config)

In [0]:
# This creates the config file for our JoeyNMT system. It might seem overwhelming so we've provided a couple of useful parameters you'll need to update
# (You can of course play with all the parameters if you'd like!)

name = '%s%s' % (source_language, target_language)
gdrive_path = os.environ["gdrive_path"]

# Create the config
config = """
name: "{name}_transformer"

data:
    src: "{source_language}"
    trg: "{target_language}"
    train: "{gdrive_path}/train.bpe"
    dev:   "{gdrive_path}/dev.bpe"
    test:  "{gdrive_path}/test.bpe"
    level: "bpe"
    lowercase: False
    max_sent_length: 100
    src_vocab: "{gdrive_path}/vocab.txt"
    trg_vocab: "{gdrive_path}/vocab.txt"

testing:
    beam_size: 5
    alpha: 1.0

training:
    # load_model: "{gdrive_path}/models/{name}_transformer/{last_checkpoint}" # uncommented to load a pre-trained model from last checkpoint
    random_seed: 42
    optimizer: "adam"
    normalization: "tokens"
    adam_betas: [0.9, 0.999] 
    scheduling: "plateau"           # TODO: try switching from plateau to Noam scheduling
    patience: 5                     # For plateau: decrease learning rate by decrease_factor if validation score has not improved for this many validation rounds.
    learning_rate_factor: 0.5       # factor for Noam scheduler (used with Transformer)
    learning_rate_warmup: 1000      # warmup steps for Noam scheduler (used with Transformer)
    decrease_factor: 0.7
    loss: "crossentropy"
    learning_rate: 0.0003
    learning_rate_min: 0.00000001
    weight_decay: 0.0
    label_smoothing: 0.1
    batch_size: 4096
    batch_type: "token"
    eval_batch_size: 3600
    eval_batch_type: "token"
    batch_multiplier: 1
    early_stopping_metric: "ppl"
    epochs: 50                     # TODO: Decrease for when playing around and checking of working. Around 30 is sufficient to check if its working at all
    validation_freq: 1000          # TODO: Set to at least once per epoch.
    logging_freq: 100
    eval_metric: "bleu"
    model_dir: "{model_temp_dir}"
    overwrite: True               # TODO: Set to True if you want to overwrite possibly existing models. 
    shuffle: True
    use_cuda: True
    max_output_length: 100
    print_valid_sents: [0, 1, 2, 3]
    keep_last_ckpts: 3

model:
    initializer: "xavier"
    bias_initializer: "zeros"
    init_gain: 1.0
    embed_initializer: "xavier"
    embed_init_gain: 1.0
    tied_embeddings: True
    tied_softmax: True
    encoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4             # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256   # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
    decoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4              # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256    # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
""".format(name=name, gdrive_path=os.environ["gdrive_path"], source_language=source_language, last_checkpoint=last_checkpoint, model_temp_dir=model_temp_dir, target_language=target_language)
with open("joeynmt/configs/transformer_{name}.yaml".format(name=name),'w') as f:
    f.write(config)

In [34]:
# Train the model
# You can press Ctrl-C to stop. And then run the next cell to save your checkpoints! 
!cd joeynmt; python3 -m joeynmt train configs/transformer_$src$tgt.yaml

Streaming output truncated to the last 5000 lines.
2020-04-10 16:36:54,255 Epoch  11 Step:    45000 Batch Loss:     1.937235 Tokens per Sec:    19127, Lr: 0.000300
2020-04-10 16:37:09,629 Example #0
2020-04-10 16:37:09,629 	Source:     " Ka dũba yadda Muke sarrafa ãyõyi , tsammãninsu sunã fahimta ! "
2020-04-10 16:37:09,629 	Reference:  Note how We explain the revelations , so that they may understand . ”
2020-04-10 16:37:09,630 	Hypothesis: See how We explain the signs that they may understand .
2020-04-10 16:37:09,630 Example #1
2020-04-10 16:37:09,630 	Source:     Dã Mun so , dã Mun mayar da shi ruwan zartsi . To don me bã ku gõdẽwa ?
2020-04-10 16:37:09,630 	Reference:  If We pleased , We would have made it salty ; why do you not then give thanks ?
2020-04-10 16:37:09,630 	Hypothesis: If We willed , We would have brought it into a rain , so why do you not give thanks ?
2020-04-10 16:37:09,631 Example #2
2020-04-10 16:37:09,631 	Source:     Kuma waɗanda suka kãfirta da ãyõyinMu , sũ

In [0]:
# Copy the created models from the temporary storage to main storage on google drive for persistant storage 
!cp -r "/content/drive/My Drive/masakhane/model-temp/"* "$gdrive_path/models/${src}${tgt}_transformer/"

In [0]:


# # Copy the created models from the notebook storage to google drive for persistant storage 
# !cp -r joeynmt/models/${src}${tgt}_transformer/* "$gdrive_path/models/${src}${tgt}_transformer/"


In [36]:
# Output our validation accuracy
! cat "$gdrive_path/models/${src}${tgt}_transformer/validations.txt"

Steps: 1000	Loss: 118695.21875	PPL: 80.54411	bleu: 0.96442	LR: 0.00030000	*
Steps: 2000	Loss: 104461.89062	PPL: 47.58518	bleu: 1.60566	LR: 0.00030000	*
Steps: 3000	Loss: 94722.87500	PPL: 33.19560	bleu: 3.77722	LR: 0.00030000	*
Steps: 4000	Loss: 88046.37500	PPL: 25.93390	bleu: 5.32723	LR: 0.00030000	*
Steps: 5000	Loss: 84236.48438	PPL: 22.52620	bleu: 6.99058	LR: 0.00030000	*
Steps: 6000	Loss: 80296.22656	PPL: 19.47217	bleu: 8.34594	LR: 0.00030000	*
Steps: 7000	Loss: 77165.25000	PPL: 17.34348	bleu: 9.86503	LR: 0.00030000	*
Steps: 8000	Loss: 75475.37500	PPL: 16.29296	bleu: 10.34419	LR: 0.00030000	*
Steps: 9000	Loss: 72819.88281	PPL: 14.76922	bleu: 12.01028	LR: 0.00030000	*
Steps: 10000	Loss: 70819.36719	PPL: 13.71617	bleu: 12.73089	LR: 0.00030000	*
Steps: 11000	Loss: 69442.17969	PPL: 13.03520	bleu: 13.75430	LR: 0.00030000	*
Steps: 12000	Loss: 68160.10938	PPL: 12.43168	bleu: 14.23732	LR: 0.00030000	*
Steps: 13000	Loss: 66636.76562	PPL: 11.75081	bleu: 14.50643	LR: 0.00030000	*
Steps: 14000	

In [37]:
# Test our model
! cd joeynmt; python3 -m joeynmt test "$gdrive_path/models/${src}${tgt}_transformer/config.yaml"

2020-04-10 22:41:22,112 Hello! This is Joey-NMT.
2020-04-10 22:41:50,027  dev bleu:  26.01 [Beam search decoding with beam size = 5 and alpha = 1.0]
2020-04-10 22:43:52,381 test bleu:  26.77 [Beam search decoding with beam size = 5 and alpha = 1.0]


In [0]:
# python3 plot_validation.py "$gdrive_path/models/${src}${tgt}_transformer/" --plot_values bleu PPL --output_path my_plot.pdf